In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.cluster import KMeans
'''
https://github.com/gouravaich/k-means-clustering-movie-ratings/blob/master/k-means%20Clustering%20of%20Movie%20Ratings.ipynb
'''
# Import the Movies dataset

def users_top_rating(top_movies, max):
    ## Get the rop rated movies - max is ammount sent in
    top_movies['counts'] = pd.Series(top_movies.count(axis=1))
    top_movies_users = top_movies.sort_values('counts', ascending=False)
    top_movies_users_selection = top_movies_users.iloc[:max, :]
    top_movies_users_selection = top_movies_users_selection.drop(['counts'], axis=1)
    
    return top_movies_users_selection

def highest_rated_movie(user_ratings, max_ratings):
    user_ratings = user_ratings.append(user_ratings.count(), ignore_index=True)
    user_ratings_sorted = user_ratings.sort_values(len(user_ratings)-1, axis=1, ascending=False)
    user_ratings_sorted = user_ratings_sorted.drop(user_ratings_sorted.tail(1).index)
    top_movies = user_ratings_sorted.iloc[:, :max_ratings]
    return top_movies

def rating_density(user_movie_ratings, n_movies, n_users):
    top_movies = highest_rated_movie(user_movie_ratings, n_movies)
    top_movies = users_top_rating(top_movies, n_users)
    return top_movies

def get_recommendation(users_id):
    
    movies = pd.read_csv('movies.csv')
    movies.head()
    ratings = pd.read_csv('ratings.csv')
    ratings.head()
    ratings = pd.merge(ratings, movies, on='movieId')
    user_ratings = ratings.loc[ratings['userId']==users_id]
    ratings_matrix = ratings.pivot_table(index='userId', columns='title', values='rating')
    user_matrix = pd.pivot_table(user_ratings, index='userId', columns= 'title', values='rating')

    user_movie_ratings =  pd.pivot_table(ratings, index='userId', columns= 'title', values='rating')
    ##Send 2000 to get the top 2000 movies
    top_3thousand = highest_rated_movie(user_movie_ratings, 3000)
    sparse_ratings = csr_matrix(pd.SparseDataFrame(top_3thousand).to_coo())
    predictions = KMeans(n_clusters=2, algorithm='full').fit_predict(sparse_ratings)
    clustered = pd.concat([top_3thousand.reset_index(), pd.DataFrame({'group':predictions})], axis=1)
    cluster_number = 1
    #number of users = 600 - some users wont be able to use this function if they have not rated enough movies
    num_users = 600
    num_movies = 1000 # Only will show the top 1000 movies
    cluster = clustered[clustered.group == cluster_number].drop(['index', 'group'], axis=1)

    cluster = rating_density(cluster, num_movies, num_users)
    #get user sent in by function
    user_id = users_id
    #user_2_ratings = cluster.iloc[cluster.index==users_id].T.squeeze()
    #user_2_ratings  = cluster.loc[user_id, :]
    ##user_2_ratings  = cluster.iloc[cluster['userId']==7]
    user_2_ratings = user_matrix
    user_2_unrated_movies =  user_2_ratings[user_2_ratings.isnull()]
    avg_ratings = pd.concat([user_2_unrated_movies, cluster.mean()], axis=1, join='inner').loc[:,0]
    result = avg_ratings.sort_values(ascending=False)[:20]
    result = result.index.tolist()
    return result
    
fart  = get_recommendation(1001)
fart

[]

In [63]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.merge(ratings, movies, on='movieId')
user_ratings = ratings.loc[ratings['userId']==1001]
user_ratings = pd.pivot_table(user_ratings, index='userId', columns= 'title', values='rating')

user_ratings

title,"13th Warrior, The (1999)",20 Dates (1998),"Abyss, The (1989)","Adventures of Robin Hood, The (1938)",Alice in Wonderland (1951),Alien (1979),All Quiet on the Western Front (1930),American Beauty (1999),American History X (1998),"American Tail, An (1986)",...,Wild Things (1998),Willow (1988),Willy Wonka & the Chocolate Factory (1971),Winnie the Pooh and the Blustery Day (1968),"Wizard of Oz, The (1939)","Wolf Man, The (1941)",X-Men (2000),Young Frankenstein (1974),Young Sherlock Holmes (1985),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1001,4.0,4.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,...,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0
